# Ceneo Scraper

1. Import bibliotek

In [ ]:
import requests
from bs4 import BeautifulSoup

2. Strona produktu

In [ ]:
product_id = '84514582'
url = f'https://www.ceneo.pl/{product_id}#tab=reviews'

3. Wysłanie żądania dostępu do zasobu

In [ ]:
response = requests.get(url)
response.status_code

4. Pobranie opinii z kodu HTML strony

In [ ]:
page = BeautifulSoup(response.text, 'html.parser')
opinions = page.select('.js_product-review')
opinion = page.select_one('.js_product-review')
print(type(opinions))
print(type(opinion))
print(opinion)

5. Pobranie składowych pojedynczej opinii

|składowa|zmienna|selektor|
|--------|-------|--------|
|identyfikator opinii|opinion_id|["data-entry-id"]|
|autora|author|.user-post__author-name|
|rekomendacje|recommandation|.user-post__author-recomendation > em|
|liczbę gwiazdek|rating|.user-post__score-count|
|treść opinii|content|.user-post__text|
|listę zalet|pros|review-feature__title--positives ~ review-feature__item|
|listę wad|cons|review-feature__title--negatives ~ review-feature__item|
|dawa wystawienia|opinion_date|.user-post__published > time:nth-child(1)["datetime"]|
|data zakupu produktu|purchase_date|.user-post__published > time:nth-child(2)["datetime"]|
|ile osób uznało opinię za przydatną|likes|button.vote-yes > span|
|ile osób uznało opinię za nieprzydatną|dislikes|button.vote-no > span|

In [ ]:
author = opinion.select_one(".user-post__author-name").get_text().strip()
#recommandation = opinion.select_one(".user-post__author-recomendation > em").get_text().strip()
rating = opinion.select_one(".user-post__score-count").get_text().strip()
content = opinion.select_one(".user-post__text").get_text().strip()
pros = opinion.select("review-feature__title--positives ~ review-feature__item")
cons = opinion.select("review-feature__title--negatives ~ review-feature__item")
opinion_date = opinion.select_one(".user-post__published > time:nth-child(1)")["datetime"].strip()
purchase_date = opinion.select_one(".user-post__published > time:nth-child(2)")["datetime"].strip()
likes = opinion.select_one("button.vote-yes > span").get_text().strip()
dislikes = opinion.select_one("button.vote-no > span").get_text().strip()
opinion_id = opinion["data-entry-id"].strip()

pros_list = [p.get_text().strip() for p in pros]
cons_list = [c.get_text().strip() for c in cons]